> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Before submitting your project, it will be a good idea to go back through your report and remove these sections to make the presentation of your work as tidy as possible. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Investigate a Dataset (Replace this with something more specific!)

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

# Gather Data

```
SELECT Match.id, 
Country.name AS country_name, 
League.name AS league_name, 
season, 
stage, 
 date,
 HT.team_long_name AS  home_team,
 AT.team_long_name AS away_team,
 home_team_goal, 
 away_team_goal
 FROM Match
 JOIN Country on Country.id = Match.country_id
 JOIN League on League.id = Match.league_id
 LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
 LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
 ORDER by date;
 
 ```

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs

%matplotlib inline


In [49]:
import sqlite3
path = "./dataset/" 
database = path + 'database.sqlite'

conn = sqlite3.connect(database)
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)

tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [50]:
df_team_goals = pd.read_sql("""SELECT Match.id, 
Country.name AS country_name, 
League.name AS league_name, 
season, 
stage, 
 date,
 HT.team_long_name AS  home_team,
 AT.team_long_name AS away_team,
 home_team_goal, 
 away_team_goal
 FROM Match
 JOIN Country on Country.id = Match.country_id
 JOIN League on League.id = Match.league_id
 LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
 LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
 ORDER by date;""", conn)
df_team_goals.head()

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,away_team_goal
0,24559,Switzerland,Switzerland Super League,2008/2009,1,2008-07-18 00:00:00,BSC Young Boys,FC Basel,1,2
1,24560,Switzerland,Switzerland Super League,2008/2009,1,2008-07-19 00:00:00,FC Aarau,FC Sion,3,1
2,24561,Switzerland,Switzerland Super League,2008/2009,1,2008-07-20 00:00:00,FC Luzern,FC Vaduz,1,2
3,24562,Switzerland,Switzerland Super League,2008/2009,1,2008-07-20 00:00:00,Neuchâtel Xamax,FC Zürich,1,2
4,24613,Switzerland,Switzerland Super League,2008/2009,2,2008-07-23 00:00:00,FC Basel,Grasshopper Club Zürich,1,0


In [51]:
df_team_goals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 10 columns):
id                25979 non-null int64
country_name      25979 non-null object
league_name       25979 non-null object
season            25979 non-null object
stage             25979 non-null int64
date              25979 non-null object
home_team         25979 non-null object
away_team         25979 non-null object
home_team_goal    25979 non-null int64
away_team_goal    25979 non-null int64
dtypes: int64(4), object(6)
memory usage: 2.0+ MB


In [52]:
df_team_goals.describe()

,id,stage,home_team_goal,away_team_goal
count,25979.000000,25979.000000,25979.000000,25979.000000
mean,12990.000000,18.242773,1.544594,1.160938
std,7499.635658,10.407354,1.297158,1.142110
min,1.000000,1.000000,0.000000,0.000000
25%,6495.500000,9.000000,1.000000,0.000000
50%,12990.000000,18.000000,1.000000,1.000000
75%,19484.500000,27.000000,2.000000,2.000000
max,25979.000000,38.000000,10.000000,9.000000


In [53]:
df_team_goals.dtypes

id                 int64
country_name      object
league_name       object
season            object
stage              int64
date              object
home_team         object
away_team         object
home_team_goal     int64
away_team_goal     int64
dtype: object

In [54]:
df_team_goals.shape

(25979, 10)

In [55]:
df_team_goals.duplicated().sum()

0

In [56]:
df_team_goals.columns

Index(['id', 'country_name', 'league_name', 'season', 'stage', 'date',
       'home_team', 'away_team', 'home_team_goal', 'away_team_goal'],
      dtype='object')

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)

In [57]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Question 1: What teams improved the most over the time period?

In [58]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


In [59]:
df_home_team_goals = df_team_goals[[ 'season', 'stage',  'home_team', 'home_team_goal' ]]
df_home_team_goals.head()

,season,stage,home_team,home_team_goal
0,2008/2009,1,BSC Young Boys,1
1,2008/2009,1,FC Aarau,3
2,2008/2009,1,FC Luzern,1
3,2008/2009,1,Neuchâtel Xamax,1
4,2008/2009,2,FC Basel,1


In [60]:
df_away_team_goals = df_team_goals[[ 'season', 'stage', 'away_team', 'away_team_goal']]
df_away_team_goals.head()

,season,stage,away_team,away_team_goal
0,2008/2009,1,FC Basel,2
1,2008/2009,1,FC Sion,1
2,2008/2009,1,FC Vaduz,2
3,2008/2009,1,FC Zürich,2
4,2008/2009,2,Grasshopper Club Zürich,0


In [61]:
df_away_team_goals.tail()

,season,stage,away_team,away_team_goal
25974,2015/2016,36,Grasshopper Club Zürich,1
25975,2015/2016,36,FC St. Gallen,0
25976,2015/2016,36,FC Sion,2
25977,2015/2016,36,BSC Young Boys,3
25978,2015/2016,36,FC Vaduz,1


In [62]:
df_home_team_goals.tail()

,season,stage,home_team,home_team_goal
25974,2015/2016,36,FC Basel,0
25975,2015/2016,36,Lugano,3
25976,2015/2016,36,FC Luzern,2
25977,2015/2016,36,FC Thun,0
25978,2015/2016,36,FC Zürich,3


In [63]:
df_home_team_goals['home_team'].nunique()

296

In [64]:
df_away_team_goals['away_team'].nunique()

296

In [65]:
df_away_team_goals['season'].nunique()

8

In [66]:
df_away_team_goals['season'].unique()

array(['2008/2009', '2009/2010', '2010/2011', '2011/2012', '2012/2013',
       '2013/2014', '2014/2015', '2015/2016'], dtype=object)

In [216]:
df_seasons_hometeam = []
df_seasons_awayteam = []
for season_goal in df_home_team_goals['season'].unique():
#     print(season_goal)
    df_hometeam_season_goal = df_home_team_goals.query('season==@season_goal')[['home_team','home_team_goal']]
    df_hometeam_season_goal = df_hometeam_season_goal.groupby('home_team').sum()
    df_awayteam_season_goal = df_away_team_goals.query('season==@season_goal')[['away_team','away_team_goal']]
    df_awayteam_season_goal = df_awayteam_season_goal.groupby('away_team').sum()
    
    df_seasons_hometeam.append(df_hometeam_season_goal)
    df_seasons_awayteam.append(df_awayteam_season_goal)
    
print(len(df_seasons_hometeam))    

8


In [217]:
df_season_total_goals = []
for i in range(len(df_seasons_hometeam)):
    
    df_seasons_hometeam[i].rename(columns={'home_team_goal': 'goals'},inplace=True)
    df_seasons_awayteam[i].rename(columns={'away_team_goal': 'goals'},inplace=True)
    df_seasons_hometeam[i].index.names = ['team_name']
    df_seasons_awayteam[i].index.names = ['team_name']
    df_seasons_hometeam[i].reset_index(level=0, inplace=True)
    df_seasons_awayteam[i].reset_index(level=0, inplace=True)
    df_season_total_goals.append(df_seasons_hometeam[i].append(df_seasons_awayteam[i],sort=False)) 
    
print(len(df_season_total_goals))

8


In [218]:
for i in range(8):
    df_season_total_goals[i] = df_season_total_goals[i].groupby('team_name').sum()
    

In [219]:
df_season_total_goals[0].sort_values(by='goals',ascending=False).head()

,goals
team_name,
FC Barcelona,105
BSC Young Boys,85
Real Madrid CF,83
Atlético Madrid,80
Celtic,80


In [220]:
df_season_total_goals[7].sort_values(by='goals',ascending=False).head()

,goals
team_name,
FC Barcelona,112
Real Madrid CF,110
Paris Saint-Germain,102
Celtic,93
FC Basel,88


In [250]:
df_season_improve = []

for i in range(7):
    df_improve = []
    df_improve = df_season_total_goals[i+1].sub(df_season_total_goals[i],fill_value=0)
    df_improve.rename(columns={'goals': 'improvement'},inplace=True)

    df_season_improve.append(df_improve)

In [251]:
for i in range(7):
    
    df_season_improve[i] = df_season_improve[i].sort_values(by='improvement', ascending=False)
    df_season_improve[i].reset_index(level=0, inplace=True)
    print(df_season_improve[i].head(10))
    print('--------------------------------------------------------------------------')

                team_name  improvement
0        St. Johnstone FC         57.0
1           FC St. Gallen         53.0
2  Montpellier Hérault SC         50.0
3                    Bari         49.0
4                   Parma         46.0
5           Real Zaragoza         46.0
6               VVV-Venlo         43.0
7                 Burnley         42.0
8                 RC Lens         40.0
9             CD Tenerife         40.0
--------------------------------------------------------------------------
                      team_name  improvement
0              Newcastle United         56.0
1          West Bromwich Albion         56.0
2                     Blackpool         55.0
3  Inverness Caledonian Thistle         52.0
4                 Real Sociedad         49.0
5          1. FC Kaiserslautern         48.0
6                       FC Thun         48.0
7                       SM Caen         46.0
8                         Lecce         46.0
9                     Excelsior         45.0
-

### Question 2: Which players had the most penalties?

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


### Question 3: What team attributes lead to the most victories?

<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!